# Extract percentage of variance explained from GEMMA log

In [1]:
library(gt)
library(tools)

In [2]:
library(stringr)
library(tidyr)
library(foreach)

<div class="alert alert-block alert-info"> Purpose: Compare results from different transformations and covariates, learning about statistical power (and Type II rates?). We will select specific transformations and covariates after reviewing these tables along with various plots produced in other notebooks or by `qtlutils`.</div>

In [4]:
length(files)

[1] 149

In [5]:
files <- files[!grepl("_boxcox_outa1", files)]

In [6]:
files <- files[!grepl("1323_cohort", files)]

In [8]:
h2_table <- foreach(file = files, .combine = "rbind") %do% {
    h2_and_se <- str_split_fixed(read.csv(file)[23:24, ], "= ", 2)[,2]
    h2_and_se_formatted <- paste0(h2_and_se[1], " +/- ", h2_and_se[2])
    
    trait <- file_path_sans_ext(file_path_sans_ext(basename(file)))
    #cat("\n")
    return(c(trait, h2_and_se[1], h2_and_se[2], h2_and_se_formatted))
}

In [9]:
colnames(h2_table) <- c("trait", "h2", "se", "h2_and_se")

In [10]:
h2_table <- as.data.frame(h2_table)

In [11]:
h2_table$h2 <- as.numeric(h2_table$h2)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [12]:
h2_table <- h2_table[order(h2_table$h2, decreasing = TRUE), ]

In [13]:
data.table::fwrite(h2_table, "sorted_h2_table.csv")

In [7]:
traits <- levels(factor(h2_table[,1]))

In [9]:
h2_table <- as.data.frame(h2_table)

In [10]:
colnames(h2_table) <- c("trait", "transformation", "covariate", "h2")

In [16]:
for(trait in traits){
    data <- h2_table[which(h2_table$trait == trait), ]

    data <- spread(data,
           key = covariate,
           value = h2)
    p <- gt(data)
    
    
    p <- 
    p %>%
    tab_header(
    title = md(trait),
    subtitle = md("Narrow-sense heritability calculated by GEMMA"))
    
    ## Save  out
    
    outname <- paste0("Results/h2/",
                      trait, "_h2table.png")
    gtsave(p, outname)
}